<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [ ]:
# version de python
import platform
platform.python_version()

In [ ]:
# la liste des packages installés
!conda list

In [ ]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [ ]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

In [ ]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

# Enregistrer une source de donnée

Upload des fichiers csv, crée un dataset à partir de ces fichiers et enregistre la référence sur le working set.

In [ ]:
from azureml.core import Dataset

dataset_name = 'titanic train dataset'
description='titanic training data'
file='./data/titanic/train.csv'
target_path='titanic-data/'

default_ds = ws.get_default_datastore()

if dataset_name not in ws.datasets:
    default_ds.upload_files(files=[file], target_path=target_path, overwrite=True, show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, file))

    # Register the tabular dataset
    try:
        tab_data_set.register(workspace=ws, 
                                name=dataset_name,
                                description=description,
                                tags = {'format':'CSV'},
                                create_new_version=True)
        
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

On peut vérifier le contenu du dataset.

In [ ]:
titanic_ds = ws.datasets.get("titanic train dataset")
titanic_ds.to_pandas_dataframe().head()

# Gestion des environnements ... (à creuser)

https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

https://docs.microsoft.com/fr-fr/azure/machine-learning/resource-curated-environments

https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/automl_env.yml

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Création d'une instance de calcul

## Création d'une aml compute instance

In [ ]:
from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

compute_name = "vm-ds3-v2"

# Verify that instance does not exist already
if compute_name in ws.compute_targets:
    print('Found existing instance, use it : ' + compute_name)
else:
    print('creating a new compute target...')
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

## Création d'un cluster

Il faut définir le nom du cluster, ainsi que les nombres de noeuds min et max.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it : ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print(compute_target.get_status().serialize())

## Pour utiliser une instance déjà créee

In [ ]:
compute_name = "aml-cluster"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    print('found compute target. just use it. ' + compute_name)
        
else :
    compute_target = 'local'
    print('compute target not found. work locally.')

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance

# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

In [4]:
import os
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace

subscription_id = os.environ.get("SUBSCRIPTION_ID", "9a159407-5de2-4a4d-a61c-89720c3bfb79") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "prepa-dp100-ressources") #Resource group

workspace_name = os.environ.get("WORKSPACE_NAME", "workspace-eun") #Nom workspace
ws1 = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

workspace_name = os.environ.get("WORKSPACE_NAME", "workspace-euw") #Nom workspace
ws2 = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

workspace_name = os.environ.get("WORKSPACE_NAME", "workspace-usae2") #Nom workspace
ws3 = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

workspace_name = os.environ.get("WORKSPACE_NAME", "workspace-usaw") #Nom workspace
ws4 = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

workspace_name = os.environ.get("WORKSPACE_NAME", "workspace-usaw2") #Nom workspace
ws5 = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

for ws in [ws1,ws2, ws3, ws4, ws5]:
    print(ws)
    for compute in ComputeTarget.list(ws):
        print(compute)
        if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
            print('try to stop compute', compute.name)
            compute.stop(show_output=True)

Workspace.create(name='workspace-eun', subscription_id='9a159407-5de2-4a4d-a61c-89720c3bfb79', resource_group='prepa-dp100-ressources')
{
  "Name": "machine1-eun",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningServices/workspaces/workspace-eun/computes/machine1-eun",
  "Workspace": "workspace-eun",
  "Location": "northeurope",
  "VmSize": "STANDARD_D2_V3",
  "State": "Stopped",
  "Tags": null
}
{
  "Name": "machine5-eun",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningServices/workspaces/workspace-eun/computes/machine5-eun",
  "Workspace": "workspace-eun",
  "Location": "northeurope",
  "VmSize": "STANDARD_D2_V3",
  "State": "Stopped",
  "Tags": null
}
{
  "Name": "machine3-eun",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningSe

{
  "Name": "machine4-usaw",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningServices/workspaces/workspace-usaw/computes/machine4-usaw",
  "Workspace": "workspace-usaw",
  "Location": "westus",
  "VmSize": "STANDARD_D2_V3",
  "State": "Stopped",
  "Tags": null
}
{
  "Name": "machine5-usaw",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningServices/workspaces/workspace-usaw/computes/machine5-usaw",
  "Workspace": "workspace-usaw",
  "Location": "westus",
  "VmSize": "STANDARD_D2_V3",
  "State": "Stopped",
  "Tags": null
}
{
  "Name": "machine3-usaw",
  "Id": "/subscriptions/9a159407-5de2-4a4d-a61c-89720c3bfb79/resourceGroups/prepa-dp100-ressources/providers/Microsoft.MachineLearningServices/workspaces/workspace-usaw/computes/machine3-usaw",
  "Workspace": "workspace-usaw",
  "Location": "westus",
  "VmSize": "STANDARD_

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)